In [24]:
#将friends following整理成文件
import os
import pickle
output_path='data/facebook/followings'
lines={}
user_info_dict={}
user_info_list=[]
char_set=set()

col=['id','URL','昵称','Education','Work','current_city','hometown',
        'contact information','Relationship','Family Members','About',
        'Favorite Quotes','Gender','Interested In','Languages']
dont_need=['#education','#{#','#}#','#living','#relationship',
            '#bio','#year-overviews','total']
useless=['@Other Places Lived','@Life Events']
rt_dict={'@Education':2,'@Work':3,'@Current City and Hometown':4,'current_city':5,
            '@Contact Information':6,'@Basic Information':11,'@Relationship':7,
             '@Family Members':8,'@About':9,'@Favorite Quotes':10,'Interested In':12,
             'Languages':13}

def check_has(x:list,y:str):
    for i in x:
        if i in y:
            return True
    return False

def parse(text):
    # print(text)
    #将每个用户的组合起来，解析出id:[url。。。]
    for k in text:
        try:
            idx=int(k[:k.find(':')])
            #开始行
            # assert idx not in lines,"重复处理关系 "+str(idx)
            #TODO 使用entity的编号区分吧，entity重复就选一个
            #TODO 过滤无效关系
            assert k.count(':')>=3
            #https://也有冒号，所以不能split
            lines[idx]=[]
        except ValueError:
            #不是开始行,不处理assert
            if 'total' in k or '#{#' in k or '#}#' in k:
                continue
            assert k.count('#')>=2,k
            lines[idx].append(k.split('#')[-1].strip())
    return lines

#把分隔符视为一样，不区分@Work和没有@等。。
def update(this_part,k):
    if check_has(useless,k):
        return -1
    for i in rt_dict:
        if i in k:
            return rt_dict[i]+1 #因为少写了url。。。


def parse_userinfo(text):
    # print(text)
    #解析出userinfo {id:{}} and [xxx]
    for k in text:
        try:
            idx=int(k[:k.find(':')])
            d=k.strip().split(':')
            username= d[2].strip() if d[2].strip() else d[1].strip()
            #开始行
            assert k.count(':')>=3
            #https://也有冒号，所以需要组合url
            url=':'.join(d[3:])
            user_info_dict[idx]={'昵称':username,'URL':url}
            user_info_list.append([idx,url,username]+['']*(len(col)-3))
            this_part=2 #当前处理的子索引，即Education

        except ValueError:
            #不是开始行,不处理assert
            if check_has(dont_need,k):
                continue
            if check_has(list(rt_dict.keys())+useless,k):
                #更新所在子索引
                this_part=update(this_part,k)
            else:
                #输入数据
                if this_part==-1:continue #不读取的数据
                user_info_list[this_part]+=k.strip()
                user_info_list[this_part]+=' '
                if col[this_part] not in user_info_dict[idx]:
                    user_info_dict[idx][col[this_part]]=''
                user_info_dict[idx][col[this_part]]+=k.strip()
                user_info_dict[idx][col[this_part]]+=' '

for path in os.listdir('data/facebook'):
    print(path)
    if not os.path.isdir(f'data/facebook/{path}') or 'following' in path:continue
    dd=os.listdir(f'data/facebook/{path}')
    name=''
    # print(path)
    for i in dd:
        if 'friend' in i:
            name=i
            break
    assert name != ''
    # print(name)
    print(f'data/facebook/{path}/{name}')
    for file_name in os.listdir(f'data/facebook/{path}/{name}'):
        # print(f'data/facebook/{path}/{name}/{file_name}')
        if 'log' in file_name:
            continue
        with open(f'data/facebook/{path}/{name}/{file_name}',encoding='utf-8-sig') as f:
            parse(f.readlines())
    #提取所有的字符，简便处理直接从文件中读取
    name=''
    for i in dd:
        if 'about' in i:
            name=i
            break
    assert name != ''
    print(f'data/facebook/{path}/{name}')
    for file_name in os.listdir(f'data/facebook/{path}/{name}'):
        # print(f'data/facebook/{path}/{name}/{file_name}')
        if 'log' in file_name:
            continue
        with open(f'data/facebook/{path}/{name}/{file_name}',encoding='utf-8-sig') as f:
            for s in f:
                char_set|=set(s.strip())
            #获得userinfo信息，直接读取为dict，不存成txt
        with open(f'data/facebook/{path}/{name}/{file_name}',encoding='utf-8-sig') as f:
            col=['id','昵称','Education','Worked','Lived','current_city','hometown',
                 'contact information','Relationship','Family Members','About',
                 'Favorite Quotes','Gender','Interested In','Languages']
            parse_userinfo(f.readlines())
#save
with open('data/facebook/char_set.txt','w',encoding='utf-8') as f:
    f.writelines([i+'\n' for i in char_set])
pickle.dump(char_set,open('data/facebook/char_set.pickle','wb'))


1-1596
data/facebook/1-1596/friends_1-2000
data/facebook/1-1596/about1-2000


IndexError: list index out of range

In [22]:
print(len(list(filter(lambda x:not x,lines.values()))))

28291
